## 2.3 自动求梯度

在深度学习中，我们经常需要对函数求梯度(gradient)。本节将介绍如何使用MXNet提供的autograd模块来自动求梯度。

In [1]:
from mxnet import autograd, nd

### 2.3.1 简单例子

我们先看一个简单例子：对函数 $y = 2\boldsymbol{x}^{\top}\boldsymbol{x}$ 求关于列向量 $\boldsymbol{x}$ 的梯度。我们先创建变量`x`，并赋初值。

In [2]:
x = nd.arange(4).reshape(4, 1)
x


[[0.]
 [1.]
 [2.]
 [3.]]
<NDArray 4x1 @cpu(0)>

为了求有关变量x的梯度，我们需要先调用attach_grad函数来申请存储梯度所需要的内存。

In [3]:
x.attach_grad()

下面定义有关变量x的函数。为了减少计算和内存开销，默认条件下MXNet不会记录用于求梯度的计算。我们需要调用record函数来要求MXNet记录与求梯度有关的计算。

In [4]:
with autograd.record():
    y = 2 * nd.dot(x.T, x)

由于x的形状为(4, 1)，y是一个标量。接下来我们可以通过调用backward函数自动求梯度。需要注意的是，如果y不是一个标量，MXNet将默认先对y中元素求和得到新的变量，再求该变量有关x的梯度。

In [5]:
y.backward()

函数 $y = 2\boldsymbol{x}^{\top}\boldsymbol{x}$ 关于$\boldsymbol{x}$ 的梯度应为$4\boldsymbol{x}$。现在我们来验证一下求出来的梯度是正确的。

In [6]:
assert (x.grad - 4 * x).norm().asscalar() == 0
x.grad


[[ 0.]
 [ 4.]
 [ 8.]
 [12.]]
<NDArray 4x1 @cpu(0)>

### 2.3.2 训练模式和预测模式

从上面可以看出，在调用record函数后，MXNet会记录并计算梯度。此外，默认情况下autograd还会将运行模式从预测模式转为训练模式。这可以通过调用is_training函数来查看。

In [7]:
print(autograd.is_training())
with autograd.record():
    print(autograd.is_training())

False
True


在有些情况下，同一个模型在训练模式和预测模式下的行为并不相同。

### 2.3.3 对Python控制流求梯度

使用MXNet的一个便利之处是，即使函数的计算图包含了Python的控制流(如条件和循环控制)，我们也有可能对变量求梯度。

考虑下面的程序，其中包含Python的条件和循环控制。需要强调的是，这里循环(while循环)迭代的次数和条件判断(if语句)的执行都取决于输入a的值。

In [8]:
def f(a):
    b = a * 2
    while b.norm().asscalar() < 1000:
        b = b * 2
    if b.sum().asscalar() > 0:
        c = b
    else:
        c = 100 * b
    return c

我们像之前一样使用record函数记录计算，并调用backward函数来求梯度。

In [9]:
a = nd.random.normal(shape=1)
a.attach_grad()
with autograd.record():
    c = f(a)
c.backward()

In [10]:
assert a.grad == c / a
a.grad


[1024.]
<NDArray 1 @cpu(0)>

### 练习参考答案

(1)在本节对控制流求梯度的例子中，把变量a改成一个随机向量或矩阵。此时计算结果c不再是标量，运行结果将有何变化？该如何分析该结果？

In [12]:
print("""-------把变量a改成3行4列的矩阵--------""")
a = nd.random.normal(shape=(3, 4))
print(a)
a.attach_grad()
with autograd.record():
    c = f(a)
c.backward()

print(a.grad)
print(c)

print("""\n------把变量a改成3行1列的列向量-------""")
a = nd.random.normal(shape=3).reshape(3, 1)
print(a)
a.attach_grad()
with autograd.record():
    c = f(a)
c.backward()

print(a.grad)
print(c)

-------把变量a改成3行4列的矩阵--------

[[ 0.5712682  -2.7579627   1.07628    -0.6141326 ]
 [ 1.8307649  -1.1468065   0.05383795 -2.5074806 ]
 [-0.59164983  0.8586049  -0.22794184  0.20131476]]
<NDArray 3x4 @cpu(0)>

[[25600. 25600. 25600. 25600.]
 [25600. 25600. 25600. 25600.]
 [25600. 25600. 25600. 25600.]]
<NDArray 3x4 @cpu(0)>

[[ 14624.466  -70603.84    27552.768  -15721.794 ]
 [ 46867.582  -29358.246    1378.2517 -64191.504 ]
 [-15146.235   21980.285   -5835.311    5153.6577]]
<NDArray 3x4 @cpu(0)>

------把变量a改成3行1列的列向量-------

[[0.35005474]
 [0.5360521 ]
 [1.5194443 ]]
<NDArray 3x1 @cpu(0)>

[[1024.]
 [1024.]
 [1024.]]
<NDArray 3x1 @cpu(0)>

[[ 358.45605]
 [ 548.91736]
 [1555.911  ]]
<NDArray 3x1 @cpu(0)>


从上面可以发现，把变量a改为一个随机的向量或矩阵时，计算结果c也为一个向量或矩阵，得到关于a的梯度也为向量或矩阵。运算结果的判断方式还是a.grad == c / a，这里的c / a为对应元素相除。

(2)重新设计一个对控制流求梯度的例子。运行并分析结果。

In [13]:
def g(a):
    b = a * 4
    while b.norm().asscalar() < 1000:
        b = b * 4
    if b.sum().asscalar() > 0:
        c = b
    else:
        c = 100 * b
    return c

a = nd.random.normal(shape=1)
a.attach_grad()
with autograd.record():
    c = g(a)
c.backward()

assert a.grad == c / a
a.grad


[1024.]
<NDArray 1 @cpu(0)>